In [23]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import numpy as np
from random import randint
import random
import time
from scipy.stats import entropy

from ortc.utils import *
from ortc.glop_v2 import glop_v2
from ortc.ortc_v2 import ortc_v2
from ortc.ortc_v1 import ortc_v1
from otc.exactOTC import exact_otc
from ortc.entropicORTC import entropic_ortc
from experiment.isomorphism import *

from otc.exactOTC import *

In [19]:
A1 = random_tree(3, 3)
n = A1.shape[0]
perm = np.random.permutation(n)
A2 = A1[np.ix_(perm, perm)]

# Get cost function
c = get_degree_cost(A1, A2)
# A1 /= np.sum(A1)
# A2 /= np.sum(A2)

# Get transition matrices
P1 = adj_to_trans(A1)
P2 = adj_to_trans(A2)

# Run algorithm
_, ortc_cost, ortc_weight = glop_v2(A1, A2, c, vertex=True)
ortc_alignment = np.sum(ortc_weight, axis=(2, 3))
_, otc_cost, _, otc_alignment = exact_otc(P1, P2, c)
print('ORTC cost: ', ortc_cost)
print('NetOTC cost: ', otc_cost)

ORTC cost:  0.0
NetOTC cost:  1.0000000000000002


In [40]:
import numpy as np
from ortools.linear_solver import pywraplp

def get_best_stat_dist(P, c):
    n = P.shape[0]
    c = c.reshape(n, -1).flatten()  
    
    # Create the linear solver with the GLOP 
    solver = pywraplp.Solver.CreateSolver('GLOP')
    
    # Decision variables: stat_dist[i]
    stat_dist = [solver.NumVar(0.0, solver.infinity(), f'stat_dist_{i}') for i in range(n)]

    # Constraints: (P' - I) * stat_dist = 0
    for i in range(n):
        constraint_expr = solver.Sum((P[j, i] - (1.0 if i == j else 0.0)) * stat_dist[j] for j in range(n))
        solver.Add(constraint_expr == 0)

    # Constraint: sum(stat_dist) == 1
    solver.Add(solver.Sum(stat_dist) == 1)

    # Create objective function
    solver.Minimize(solver.Sum(stat_dist * c))

    # Solve the problem.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        stat_dist_values = np.array([var.solution_value() for var in stat_dist])
        exp_cost = solver.Objective().Value()
        return stat_dist_values, exp_cost
    else:
        # In case the solver fails, try rescaling constraints.
        alpha = 1
        while alpha >= 1e-10:
            alpha /= 10
            # Create a new solver instance to reset all variables and constraints.
            solver = pywraplp.Solver.CreateSolver('GLOP')

            # Decision variables: stat_dist[i]
            stat_dist = [solver.NumVar(0.0, solver.infinity(), f'stat_dist_{i}') for i in range(n)]

            # Constraints: alpha * (P' - I) * stat_dist = 0
            for i in range(n):
                constraint_expr = solver.Sum(alpha * (P[j, i] - (1.0 if i == j else 0.0)) * stat_dist[j] for j in range(n))
                solver.Add(constraint_expr == 0)

            # Constraint: alpha * sum(stat_dist) == alpha
            solver.Add(solver.Sum([alpha * var for var in stat_dist]) == alpha)

            # Create objective function
            solver.Minimize(solver.Sum(stat_dist * c))

            # Solve the problem again.
            status = solver.Solve()
            if status == pywraplp.Solver.OPTIMAL:
                stat_dist_values = np.array([var.solution_value() for var in stat_dist])
                exp_cost = solver.Objective().Value()
                return stat_dist_values, exp_cost

        # If still no solution, raise an error.
        raise ValueError('Failed to compute stationary distribution.')


In [39]:
get_best_stat_dist(P, c)

(array([0.   , 0.5  , 0.   , 0.125, 0.   , 0.125, 0.125, 0.   , 0.125]), 0.0)

In [41]:
get_best_stat_dist(P, c)

(array([0.   , 0.5  , 0.   , 0.125, 0.   , 0.125, 0.125, 0.   , 0.125]), 0.0)